# Deeper Exploration of The Graph

My next experiment will be to write code that can query $n$ transactions from uniswap subgraph, get the addresses of the accounts that performed those transactions and use them to get the specific account activity. Then, count the number of transactions they've performed over the last week, iterate on that until I've got a decent amount of accounts' weekly transaction numbers and visualize that distribution... I would expect a skewed distribution where bots have a significantly higher amount of transactions than most accounts but we'll see.  
I can use that to determine a naive "threshold" to cut off hft bots from more interesting activity... then drill down on those more interesting accounts.   
I think the next step then would be to figure out how to evaluate that weekly performance. If we can sort on some metric of success (maybe account % increase?) then we might be able to cherry pick a few exemplary accounts demonstrating some of the "pause", "play", or "shuffle" strategies.

In [1]:
import requests
import json
from web3 import Web3

# pip install etherscan-python
from etherscan import Etherscan

GRAPH_REQUEST_TIMEOUT = 10000
with open('api_key.json', mode='r') as key_file:
    API_KEY = json.loads(key_file.read())['key']
    
eth = Etherscan(API_KEY)

infura_url = "https://mainnet.infura.io/v3/9c8c7a4e9f85467eb1e298ab3431659f"
web3 = Web3(Web3.HTTPProvider(infura_url))

In [2]:
# query n transactions from the graph
def uniswap_transactions(count):
    print("getting {} uniswap transactions".format(count))
    headers = {}
    query = """
    {
    transactions(first: %(count)s, orderBy:timestamp, orderDirection:desc) {
      id
      blockNumber
      timestamp
      mints {
        id
      }
      burns {
        id
      }
      swaps {
        id
      }
    }
    }
    """ % { 'count': count }
    
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                           json={'query':query}, headers=headers, timeout=GRAPH_REQUEST_TIMEOUT)
    if request.status_code == 200:
        print("done")
        return request.json()

In [3]:
# get the addresses of those accounts
def getAddresses(txns):
    # first need to get the txn ids
    ids = [x['id'] for x in txns['data']['transactions']]
    # then get the wallet addresses in those txn ids
    addresses = [web3.eth.getTransaction(tx)['from'] for tx in ids]   # returns the receipt, parse the 'from' field
    return addresses

In [4]:
txns = uniswap_transactions(10)
addresses = getAddresses(txns)

getting 10 uniswap transactions
done


In [5]:
addresses

['0x9f5ae9A77c709754c5eC26e7b0ac34061eE6D69c',
 '0x1D86C0da994Eff5132fe1f8c1Ea0F6b1eE2f2B8d',
 '0xEeeEeeeec58706fF86d18BE81296FdDC7A96934a',
 '0x288457a12496cbaF6e51Aa4F306dA1DC6ddeBF1D',
 '0x4a420e0878b63775FF4f4DB22B4cEf3439714693',
 '0x30653411Eb9F2560883c9BA1E2133d66a5610936',
 '0x2D35eD31D43d6bF5CCc5D7A085fE1eA9829A64D1',
 '0x32Cd1F7BdaE90B87E9b58d25d2ef7b09f6A9B548',
 '0xD986866Fbba56555D06d7a8FD2E1e22eA4B34A50',
 '0xE928a2d1fe385d776fD448Ec5D5F85ED0E8F07Bb']

In [38]:
# get each address's weekly transactions -- use etherscan api package
# NOTE: returns the last 10k events
# @params: address (str), startblock (int), endblock (int), sort (str)
eth.get_normal_txs_by_address(addresses[0], 11870000, 11918179, 'desc')    # block difference approximately a week

5

In [ ]:
# create pandas df with < address | external_tx_count | startblock | endblock >